In [1]:
import os
from surfer import Brain, project_volume_data
import nibabel as nib
import numpy as np
import pandas as pd
os.chdir('/Users/mengdu/Repos/hierarchy/')

In [2]:
nav_p_df = pd.read_csv('nav_delta-sim_ttest.csv', index_col=0)
corrected_nav_p = nav_p_df['p_fdr_bh']
nav_p = {}
nav_p['lh'] = corrected_nav_p.loc[[parcel for parcel in corrected_nav_p.index if '-lh-' in parcel or '-Left-' in parcel]]
nav_p['lh'].index = [parcel.split('-')[-1] for parcel in nav_p['lh'].index]
nav_p['rh'] = corrected_nav_p.loc[[parcel for parcel in corrected_nav_p.index if '-rh-' in parcel or '-Right-' in parcel]]
nav_p['rh'].index = [parcel.split('-')[-1] for parcel in nav_p['rh'].index]


In [4]:
sac_p_df = pd.read_csv('sac_delta-sim_ttest.csv', index_col=0)
corrected_sac_p = sac_p_df['p_fdr_bh']
sac_p = {}
sac_p['lh'] = corrected_sac_p.loc[[parcel for parcel in corrected_sac_p.index if '-lh-' in parcel or '-Left-' in parcel]]
sac_p['lh'].index = [parcel.split('-')[-1] for parcel in sac_p['lh'].index]
sac_p['rh'] = corrected_sac_p.loc[[parcel for parcel in corrected_sac_p.index if '-rh-' in parcel or '-Right-' in parcel]]
sac_p['rh'].index = [parcel.split('-')[-1] for parcel in sac_p['rh'].index]


In [5]:
hemi = 'lh'
labels, ctab, names = nib.freesurfer.read_annot(hemi + '.aparc.DKTatlas40.annot')
nav_data = np.zeros(labels.shape[0]) - 1
sac_data = np.zeros(labels.shape[0]) - 1
roi_overlap = set()
names = [name.decode("utf-8") for name in names]
for roi in names:
    if roi in nav_p[hemi]:
        roi_overlap.add(roi)
        nav_data[labels == names.index(roi)] = nav_p[hemi][roi]
    if roi in sac_p[hemi]:
        roi_overlap.add(roi)
        sac_data[labels == names.index(roi)] = sac_p[hemi][roi]
    else:
        print(roi)
print(roi_overlap)

unknown
bankssts
corpuscallosum
frontalpole
temporalpole
{'parsorbitalis', 'paracentral', 'inferiortemporal', 'lateralorbitofrontal', 'fusiform', 'rostralanteriorcingulate', 'posteriorcingulate', 'parstriangularis', 'superiorfrontal', 'transversetemporal', 'lingual', 'inferiorparietal', 'middletemporal', 'superiortemporal', 'supramarginal', 'cuneus', 'parsopercularis', 'precuneus', 'lateraloccipital', 'caudalmiddlefrontal', 'rostralmiddlefrontal', 'pericalcarine', 'superiorparietal', 'parahippocampal', 'caudalanteriorcingulate', 'precentral', 'insula', 'postcentral', 'isthmuscingulate', 'medialorbitofrontal', 'entorhinal'}


In [12]:
# Show visualization
%gui qt

brain = Brain('fsaverage', hemi, 'inflated', alpha=1,
              background='white', cortex='low_contrast',
              views=['lateral'], show_toolbar=True)


In [13]:
# Add borders
class Label(object):
    def __init__(self, index):
        self.hemi = hemi
        self.name = ''
        self.vertices = np.where(labels == index)[0]

for i in range(len(names)):
    brain.add_label(Label(i), borders=1, color='#222222', hemi=hemi, alpha=.5)

In [14]:
p_level = .01
nav_data_thr = np.copy(nav_data)
sac_data_thr = np.copy(sac_data)
nav_roi_thr = np.isin(names, nav_p[hemi][nav_p[hemi] < p_level].index)
sac_roi_thr = np.isin(names, sac_p[hemi][sac_p[hemi] < p_level].index)
nav_data_thr[~np.isin(labels, np.where(nav_roi_thr))] = -1
sac_data_thr[~np.isin(labels, np.where(sac_roi_thr))] = -1
brain.add_data(nav_data_thr, min=0, max=0.8, thresh=0, colormap='Reds_r', alpha=.7)
brain.add_data(sac_data_thr, min=0, max=0.8, thresh=0, colormap='GnBu_r', alpha=.6)


colormap sequential: [0.00e+00, 4.00e-01, 8.00e-01] (opaque)
colormap sequential: [0.00e+00, 4.00e-01, 8.00e-01] (opaque)


In [19]:
brain.remove_data()